In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tokenization
import collections
import ast
import time
from nltk.tokenize import TweetTokenizer

In [2]:
columns = ['character1','character2','movietitle','conversation']

movie_conver = []
with open('data/movie_conversations.txt','r') as file:
    for l in file:
        lines = l.split('+++$+++')
        line_content =  eval(lines[-1].rstrip())
        lines = [x.strip() for x in lines[:-1]]
        lines.append(line_content)
        movie_conver.append(lines)
movie_conver = pd.DataFrame(movie_conver, columns=columns)

In [3]:
columns = ['line','character_id','movietitle','character_name','text']

movie_lines = []
with open('data/movie_lines.txt','r', errors='ignore') as file:
    for l in file:
        lines = l.split('+++$+++')
        lines[-1] = lines[-1].rstrip()
        lines = [x.strip() for x in lines]
        movie_lines.append(lines)
movie_lines = pd.DataFrame(movie_lines, columns=columns)

In [4]:
assert len(movie_lines['line'].unique()) == len(movie_lines['line'])
movie_lines.set_index('line',inplace=True)

In [5]:
movie_title = movie_conver['movietitle'].unique()
select_title = movie_title[1:10]

select_movie_conver = movie_conver[np.isin(movie_conver['movietitle'], select_title)]
select_movie_lines = movie_lines[np.isin(movie_lines['movietitle'], select_title)]

In [6]:
class Example():
    
    def __init__(self,input_sentences, output_sentences):
        
        self.input_sentences = input_sentences
        self.output_sentences = output_sentences
               

In [7]:
def convert_toIndex(tokens, word_dict):

    index_features = []
    for token in tokens:
        if not token in word_dict:
            word_dict[token] = len(word_dict) + 1

        index_features.append(word_dict[token])
    return index_features

In [8]:
def truncate_sequence(text_a, text_b, max_len):
    
    while len(text_a) + len(text_b) + 3 >= max_len:
        if len(text_b) > len(text_a):
            text_b.pop()
        else:
            text_a.pop()

In [9]:
def convert_toExample(num_i, conversation, tokenizer, max_input_seq, max_output_seq, word_dict):
#     convert text into examples
    
    num_convers = len(conversation)
    for i, _ in enumerate(conversation):
        
        if i < num_convers - 1:
            num_i += 1
            
            input_texts = tokenizer.tokenize(conversation[i])
            response = tokenizer.tokenize(conversation[i+1])
#             response = tokenizer.tokenize(conversation[i+2])          
#             truncate_sequence(text_q, text_p, max_input_seq)              
#             input_texts = text_q + ['[SEQ]'] + text_p
            if len(input_texts) == 0 or len(response) == 0:
                continue

            if len (input_texts) > max_input_seq:
                input_texts = input_texts[:max_output_seq]
            
            if len (response) > max_output_seq - 2:
                response = response[:max_output_seq - 2]
                
            response = response + ['[END]']
        
            input_id = convert_toIndex(input_texts, word_dict)
            output_id = convert_toIndex(response, word_dict)
            
            if len(input_id) < max_input_seq:
                input_id.extend([0] * (max_input_seq - len(input_id)))
                
            if len(output_id) < max_output_seq:
                output_id.extend([0] * (max_output_seq - len(output_id)))
                
            assert len(input_id) == max_input_seq
            assert len(output_id) == max_output_seq
            
            if num_i % 5000 == 0:
                tf.logging.info('input text: %s' % ' '.join(input_texts))
                tf.logging.info('input id: %s' % ' '.join([str(x) for x in input_id]))
                tf.logging.info('output text: %s' % ' '.join(response))
                tf.logging.info('output id: %s' % ' '.join([ str(x) for x in output_id]))
            
            yield num_i, Example(input_id, output_id)
        

In [10]:
def build_dataSet(movie_conver, movie_lines, max_input_seq, max_output_seq, tokenizer):
    
    word_dict = collections.OrderedDict()
    word_dict['#UNTOKEN#'] = 0
    word_dict['[START]'] = 1
    num_i = 0
    
    examples = []
    for index, row in movie_conver.iterrows():
        line_index = row['conversation']
        conversation = movie_lines.loc[line_index,'text']
        
        if len(conversation) > 2:
            for num_i, example in convert_toExample(num_i, conversation, tokenizer, max_input_seq, max_output_seq, word_dict):
                examples.append(example)
    return word_dict, examples
    

In [11]:
def save_data(examples, file_path):
    writer = tf.python_io.TFRecordWriter(file_path)
    
    for example in examples:
        features = collections.OrderedDict()
        features["input_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=example.input_sentences))
        features["output_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=example.output_sentences))

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()
    

In [12]:
# tokenizer = tokenization.FullTokenizer('data/vocab.txt')
tokenizer = TweetTokenizer()

In [13]:
max_input_seq = 20
max_output_seq = 20

word_dict, examples = build_dataSet(select_movie_conver, select_movie_lines, max_input_seq, max_output_seq, tokenizer)
print(len(examples))

1882


In [14]:
from sklearn.model_selection import train_test_split
train_example, val_examples = train_test_split(examples, test_size = 0.2, random_state = 3)
save_data(train_example, 'data/train')
save_data(val_examples, 'data/val')

In [15]:
del movie_conver, select_movie_conver
del movie_lines, select_movie_lines
del train_example, val_examples, examples

In [16]:
import json
with open('data/vocab.json', 'w') as fp:
    json.dump(word_dict, fp)

In [89]:
def load_DataSet(input_file, input_length, output_length, batch_size = 32, training = True):
    
    feature_description = {
      "input_ids": tf.FixedLenFeature([input_length], tf.int64),
      "output_ids": tf.FixedLenFeature([output_length], tf.int64),
      }
    
    def _parse_function(record):
        return tf.parse_single_example(record, feature_description)
    
    
    data = tf.data.TFRecordDataset(input_file)
    data = data.map(lambda x: _parse_function(x))
    data = data.cache()
    
    if training:
        data = data.shuffle(buffer_size=1000)
        
    data = data.batch(batch_size, drop_remainder=True)
    
    return data

In [90]:
def encoder(input_ids, embeding, vocab_size, embeding_size):
    
    with tf.variable_scope('encoder'):
    
        input_embeding = tf.nn.embedding_lookup(embeding, input_ids)

        rnn_layers = [tf.nn.rnn_cell.GRUCell(size) for size in [512,256]]

        multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

        sequence_length = tf.reduce_sum(tf.cast(input_ids > 0, dtype = tf.int32), axis=-1)

        outputs, state = tf.nn.dynamic_rnn(multi_rnn_cell, input_embeding, dtype=tf.float32, sequence_length = sequence_length)
    
    return outputs, state, sequence_length
    

In [91]:
def attention(attention_unit, encoder_output, sequence_length):
    
    attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(attention_unit, memory = encoder_output,
                                                               memory_sequence_length=sequence_length)
    return attention_mechanism

In [92]:
def decoder(output_ids, embeding, vocab_size, sequence_length, attention_mechanism, padding_size = 20, reuse = None, training = True):

    with tf.variable_scope('decoder', reuse = reuse):
        
        batch_size = tf.shape(output_ids)[0]
        
        rnn_layers = [tf.nn.rnn_cell.GRUCell(size) for size in [512,256]]

        multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

#         attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(attention_unit, encoder_output, encoder_length)

        attention_cell = tf.contrib.seq2seq.AttentionWrapper(multi_rnn_cell, attention_mechanism, attention_layer_size = None)

        attention_cell = tf.contrib.rnn.OutputProjectionWrapper(
                attention_cell, vocab_size, reuse=reuse)

        initial_state = attention_cell.zero_state(dtype = tf.float32, batch_size = batch_size)
        initial_state = initial_state.clone(cell_state=encoder_state)
        
        if training:
            
            input_embeding = tf.nn.embedding_lookup(embeding, output_ids)
            
            helper = tf.contrib.seq2seq.TrainingHelper(
                        inputs = input_embeding,
                        sequence_length = sequence_length,
                        )
        else:
            helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                    embeding, start_tokens=tf.zeros([batch_size], dtype=tf.int32) + word_dict['[START]'], 
                    end_token=word_dict['[END]'])

        decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = attention_cell,
                helper = helper, 
                initial_state = initial_state)

        decoder_outputs, _, decoder_length = tf.contrib.seq2seq.dynamic_decode(decoder, maximum_iterations=output_length)
        
#         padding_need = padding_size - tf.reduce_max(decoder_length)
#         padding_tf = tf.zeros([batch_size, padding_need, vocab_size], dtype = tf.float32)
#         decoder_rnn = tf.concat([decoder_outputs.rnn_output, padding_tf], axis=1)
    
    return decoder_outputs.rnn_output
    

In [93]:
def sequence_loss(prediction, label, decoder_length):
    
    one_hot_label =tf.one_hot(label, depth = tf.shape(prediction)[-1])
    per_example_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = one_hot_label, logits = prediction)   
    per_example_loss = per_example_loss * decoder_length
    
#     per_example_loss = tf.where(decoder_length < 1, tf.stop_gradient(per_example_loss), per_example_loss)
    per_example_loss = tf.reduce_sum(per_example_loss, axis = -1) / tf.cast(tf.reduce_sum(decoder_length, axis = -1), dtype = tf.float32)
    
    return per_example_loss


In [94]:
import json
with open('data/vocab.json', 'r') as fp:
    word_dict = json.load(fp)

In [106]:
tf.reset_default_graph()
train_file = 'data/train'
val_file = 'data/val'
input_length = 20
output_length = 20
batch_size = 32

embeding_size = 300
vocab_size = len(word_dict)
attention_unit = 32

train_data = load_DataSet(train_file, input_length, output_length, batch_size)
val_data = load_DataSet(val_file, input_length, output_length, batch_size, training = False)

iter_ = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)

train_init_op = iter_.make_initializer(train_data)
test_init_op = iter_.make_initializer(val_data)
features = iter_.get_next()

input_ids = features['input_ids']
output_ids = features['output_ids']

decoder_length = tf.count_nonzero(output_ids, axis = -1, dtype = tf.int32)
output_ids = tf.slice(output_ids, [0,0], [batch_size, tf.reduce_max(decoder_length, axis = -1)])

decoder_length_mask = tf.cast(tf.sequence_mask(decoder_length), dtype = tf.float32)

batch_size = tf.shape(output_ids)[0]
start_tokens = tf.zeros([batch_size], dtype=tf.int64) + word_dict['[START]']
train_output_ids = tf.concat([tf.expand_dims(start_tokens, 1), output_ids], 1)

embeding = tf.get_variable(name = 'word_embedding', shape = [vocab_size, embeding_size], dtype = tf.float32,
                                  initializer=tf.contrib.layers.xavier_initializer())

encoder_output, encoder_state, encoder_length = encoder(input_ids, embeding, vocab_size, embeding_size)

attention_mechanism = attention(attention_unit, encoder_output, encoder_length)
    
decoder_outputs = decoder(train_output_ids, embeding, vocab_size, decoder_length, attention_mechanism, reuse = None, training = True)
# infer_outputs = decoder(train_output_ids, embeding, vocab_size, decoder_length, attention_mechanism, reuse = True, training = False)

# prediction = tf.argmax(infer_outputs, axis = -1)
prediction = tf.argmax(decoder_outputs, axis = -1)

# per_example_loss = tf.contrib.seq2seq.sequence_loss(logits = decoder_outputs, targets = output_ids, weights = decoder_length_mask,  average_across_batch=False)
per_example_loss = sequence_loss(decoder_outputs, output_ids, decoder_length_mask)

loss, loss_update = tf.metrics.mean(values=per_example_loss)
total_cost = tf.reduce_mean(per_example_loss)

train_op = tf.train.AdamOptimizer().minimize(total_cost)

accuracy, accu_update = tf.metrics.accuracy(prediction, output_ids, weights = decoder_length_mask)

In [108]:
epoch = 1000
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(epoch):
        begin = time.time()
        sess.run(tf.local_variables_initializer())
        sess.run(train_init_op)
        try: 
            while True:
                _, _, los = sess.run([train_op, loss_update, loss])
        except tf.errors.OutOfRangeError:
            pass
        print('loss in traing set at epoch %s is %s' % (i, los))
        sess.run(test_init_op)
        try: 
            while True:
                _, accura= sess.run([accu_update, accuracy], options = run_opts)
        except tf.errors.OutOfRangeError:
            pass
        print('accuracy in val set at epoch %s is %s' % (i, accura))
        print('time cost for % s ' % (time.time() - begin))

[[   1    3  540    3  765  231   21  316  185  507  186  208   58  193
    42    7  101  132 2097   22]
 [   1    3  744  293 2415 1675  293 2416 1675  251 2415 2416 1675  383
   475 1048   39  214  215   22]
 [   1  215  740   21   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1 3279   45  169   19 3280  412   21   22    0    0    0    0    0
     0    0    0    0    0    0]
 [   1    3 1496 1523   67  245 1524   41   24  917   21   22    0    0
     0    0    0    0    0    0]
 [   1   63 1091   29   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  169  482   19 1451   32 1452 1453 1454 1455  263 1456  263  115
  1457   45   22    0    0    0]
 [   1    3   58  540  238  160   42  702  210    7   41 1731   57   21
  1732   42  796   19 1733   22]
 [   1 1275  829   21   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  667  946  947  829   61  948  242   55  949  27

[[   1  283   82  716   95 1081   29   63  414  507   12   34 1309  820
    55 1672   41  127  238   22]
 [   1 1275  829   21   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  197 1675  730   39   67   55 1676 1657   45   22    0    0    0
     0    0    0    0    0    0]
 [   1  407  181   45   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1    3  458   39  170   21   22    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  420   99  186  122  112  421    9   22    0    0    0    0    0
     0    0    0    0    0    0]
 [   1    3  538  356   42  198  181  831   21   22    0    0    0    0
     0    0    0    0    0    0]
 [   1  830   21   22    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1   10  171   42 1460   21    3  540  120 1702   24   55 1783   12
    19 1541 1542   21   22    0]
 [   1   63  649   62   41  558   26  593   42  702   9

[[   1  615  162   29    3   59   41  511   19  121  122   69  561   21
    22    0    0    0    0    0]
 [   1 2543  408 1100   21  417  185   39  181   42   39  734 1804   62
   769   41  716  600  120   22]
 [   1 1132   19 1131   21   22    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  604   45 2659   62  101   21   22    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  842   45   63  843   55  794   41   55  844   45    9   22    0
     0    0    0    0    0    0]
 [   1  148  122  368  181  641    7   21   63  221   11 1144  374  196
   238 1392   21   22    0    0]
 [   1  283    4  290  122   29  291  292  293  294    9   22    0    0
     0    0    0    0    0    0]
 [   1  297   42    3  408   12   19 2264  177  876   34  803  275  132
   774  507  208  389  278   22]
 [   1  111   39   38  154  323   21   22    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  197   42   39  820   41 2024  208  286   55  31

[[   1 1324 1675   12  680  145   19 2522   32   19 2523 2524 1675  145
    19 1551 1675  145   19   22]
 [   1  474 1702   24   21  169 3041 3216  101 3201  282 3202   99   42
    34  215 2001  214    9   22]
 [   1 2730 2731 2732  408  561 1888   39 1246  593   41   67  231 2644
    21   22    0    0    0    0]
 [   1  579  737 1385   21  413  540    3  194 1091   26   90 1508   21
   474  160   41  720   42   22]
 [   1 2891   21   22    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1    3  334   41  193  470   26  651   21    3  334   41  193  470
    26  652   21  376  213   22]
 [   1  215   90   12    8  818  814  641 1988   21  215   90  230 1989
   862  818  814  238 1990   22]
 [   1  886  887   21   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1 2597   38 1037 2598 2599   32  649   21  494   39  177   26  275
    21    3 1546  208   82   22]
 [   1  220 1169   21   22    0    0    0    0    0    

[[   1   63 1702   90 3201 1015 3202  356   21  169  486  372    7   26
   658   42   41  818 3225   22]
 [   1  699   42  747   42  762   21  413 1114   62   21   22    0    0
     0    0    0    0    0    0]
 [   1  148   42  367   39   29   34  215  270  522    3  171  436   24
    17   21   22    0    0    0]
 [   1  534 1490   21   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1    3 2686  263 3144   19 3145   21 1841   59   41 1108   39   42
  1112  193   21 1324   15   22]
 [   1   63   59   62   41  286  689  120   34 2020 1839 2772   45   22
     0    0    0    0    0    0]
 [   1   15   68   99  120   21    3  193  238    9   22    0    0    0
     0    0    0    0    0    0]
 [   1  982   21   22    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1 2606 3105   21   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1 1513   42 1775 1776  972   21 1777   21  183   2

...

[[   1    3 2759  976   19 1020   34 2760    8  122  278   29   63  193
   374  238   99   21   22    0]
 [   1    3  178   21   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1   63    4  386   21   63  386  145   19  573  177  574   32  507
    42  296   45   22    0    0]
 [   1  570   21   22    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  110  818  689   21  818  646  510  177   39   21   22    0    0
     0    0    0    0    0    0]
 [   1   50   12   21  447  155   24  288 2602   21   22    0    0    0
     0    0    0    0    0    0]
 [   1  890 1040  145   39   67   42   39 3063  704  455 3064   21   22
     0    0    0    0    0    0]
 [   1  495   99   21    3  396  102  778   29 3332   29   99 1655   41
   129    9   22    0    0    0]
 [   1  407   39  324  193  231   21   22    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  589   21  596  238  597   21  598  107   62   2

[[   1  635   21  277  208   67   55  974  214   45  857   52  162   41
    67   55  974   21  215   22]
 [   1  131 1291  218   39 1425 2977  573  470   42  185   62   42  702
     3   67   41  224   21   22]
 [   1  376   90   45   22    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  635   42   39  221 1406   48   12 1407 1125   21   22    0    0
     0    0    0    0    0    0]
 [   1  514  211   45   54  202   45   22    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1 1287   45   22    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  509   45   22    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1    3  465  744   41   29   22    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1   87   42 1117   42 1658  238   99   42 1040 1659   90 1660 1502
    21  646   99   19  307   22]
 [   1  664   42  215 1356   21  215   90  455 1357    

[[   1 1555    9    9 1555   57   39   34   26 3112 1797 3113    8  839
  1071  522  497   62   21   22]
 [   1   37   39   38  444   99   39   58  193  181    8 1675  308 1675
    99   21   22    0    0    0]
 [   1 2246   29   22    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1    3   82  385   39   42  201   42  702   39  884  422   62  689
    21    3  820   41   24   22]
 [   1  262   99  282  242   45   22    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  215  270    3   58   29   22    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1 3054  238  689   45    9   45    9  417   24  590  472    3 2857
   238  689   21   15  121   22]
 [   1 3308 3201  567 3202   45   22    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  197 1675  730   39   67   55 1676 1657   45   22    0    0    0
     0    0    0    0    0    0]
 [   1   65   58  193   57  305   21  534  196  154 108

[[   1  197   42  215   12  459 1319  522   21   37  324   39   77   41
  2842  689   34  665   62   22]
 [   1 1634  689   41   19  947  242 1219   21   22    0    0    0    0
     0    0    0    0    0    0]
 [   1  215  738   41  185   39   21   22    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   1  635   42 1309  820  858   41  324  107    8   29  414   48  273
    29   22    0    0    0    0]
 [   1 2730 2731 2732  408  561 1888   39 1246  593   41   67  231 2644
    21   22    0    0    0    0]
 [   1    3 1496 1523   67  245 1524   41   24  917   21   22    0    0
     0    0    0    0    0    0]
 [   1  553   12 1450   42  198 1097   21   22    0    0    0    0    0
     0    0    0    0    0    0]
 [   1   63   58  193   21  215  740   21    3  408  741   21  220   90
    62   21    3  742   55   22]
 [   1   87   29  197   42    3 1546   19 1600  820  496 1694   21   22
     0    0    0    0    0    0]
 [   1 1766 1262  192  497  208  273  230 1607   42   1

KeyboardInterrupt: 